In [175]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
%matplotlib inline

In [176]:
df=pd.read_csv('StudentsPerformance.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [177]:
x=df.drop(columns=['math score'],axis=1)

In [178]:
x.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [179]:
y=df['math score']

In [180]:
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math score, dtype: int64

In [181]:
num_features=x.select_dtypes(exclude='object').columns
cat_features=x.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
newtansform=OneHotEncoder()
ohtransform=StandardScaler()

processor=ColumnTransformer(
[
    ("OneHotEncoder",newtansform,cat_features),
    ("StandardScaler",ohtransform,num_features)
    ]
)

In [182]:
x=processor.fit_transform(x)

In [183]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [184]:
x_test.shape

(200, 19)

In [185]:
x_train.shape

(800, 19)

In [186]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
def evauate(true,predicted):
    mse = mean_squared_error(true,predicted)
    mae= mean_absolute_error(true,predicted)
    r2_scor= r2_score(true,predicted)
    return mse,mae,r2_scor

In [187]:
models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'RandomForestRegressor': RandomForestRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'XGBRegressor': XGBRegressor(),
}

model_list = []
evaluation_results = []

for model_name, model in models.items():
    model.fit(x_train, y_train)
    
    y_train_predict = model.predict(x_train)
    y_test_predict = model.predict(x_test)
    
    train_mse, train_mae, train_r2 = evauate(y_train, y_train_predict)
    test_mse, test_mae, test_r2 = evauate(y_test, y_test_predict)
    
    model_list.append(model)
    evaluation_results.append({
        'model_name': model_name,
        'train_mse': train_mse,
        'train_mae': train_mae,
        'train_r2': train_r2,
        'test_mse': test_mse,
        'test_mae': test_mae,
        'test_r2': test_r2
    })

for result in evaluation_results:
    print(f"Model: {result['model_name']}\n")
    print("The performence of the traing data:\n")
    print(f"Mean_squred erroer: {result['train_mse']}\nMean_absolute error: {result['train_mae']}\nR2: {result['train_r2']}\n")
    print("The performence of the test data:\n")
    print(f"Mean_squred erroer: {result['test_mse']}\nMean_absolute error: {result['test_mae']}\nR2: {result['test_r2']}")
    print("----------------------------------------------------")

Model: LinearRegression

The performence of the traing data:

Mean_squred erroer: 28.4013037109375
Mean_absolute error: 4.271484375
R2: 0.874022530822051

The performence of the test data:

Mean_squred erroer: 29.43263671875
Mean_absolute error: 4.2221875
R2: 0.8790464771743793
----------------------------------------------------
Model: Lasso

The performence of the traing data:

Mean_squred erroer: 43.47840400585577
Mean_absolute error: 5.206302661246526
R2: 0.8071462015863458

The performence of the test data:

Mean_squred erroer: 42.5064168384116
Mean_absolute error: 5.157881810347763
R2: 0.8253197323627853
----------------------------------------------------
Model: Ridge

The performence of the traing data:

Mean_squred erroer: 28.337788233082442
Mean_absolute error: 4.26498782372598
R2: 0.8743042615212909

The performence of the test data:

Mean_squred erroer: 29.05627219234824
Mean_absolute error: 4.211100688014257
R2: 0.8805931485028741
------------------------------------------

In [188]:
# Convert evaluation results into a DataFrame
df_results = pd.DataFrame(evaluation_results)
# Sort the DataFrame by 'test_r2' in descending order
df_results.sort_values(by='test_r2', ascending=False)

,model_name,train_mse,train_mae,train_r2,test_mse,test_mae,test_r2
2,Ridge,28.337788,4.264988,0.874304,29.056272,4.211101,0.880593
0,LinearRegression,28.401304,4.271484,0.874023,29.432637,4.222188,0.879046
4,AdaBoostRegressor,32.912846,4.724092,0.854011,36.130111,4.690940,0.851523
3,RandomForestRegressor,5.158409,1.810409,0.977119,36.511850,4.663863,0.849954
7,XGBRegressor,1.014616,0.687467,0.995500,41.903708,5.057731,0.827797
1,Lasso,43.478404,5.206303,0.807146,42.506417,5.157882,0.825320
6,KNeighborsRegressor,32.595700,4.517500,0.855418,52.683400,5.637000,0.783497
5,DecisionTreeRegressor,0.078125,0.018750,0.999653,63.675000,6.315000,0.738327


In [189]:
regression=LinearRegression(fit_intercept=True)
REG